In [2]:
from network2 import *
from dataset import *
from utils2 import *
from loss import *
from load import *

import os
os.environ['TOKENIZERS_PARALLELISM']='false'
import json
import torch
import pandas as pd
from copy import copy
from collections import defaultdict, OrderedDict
from pathlib import Path
from tqdm import tqdm

from torch.utils.data import Dataset, DataLoader

from transformers import BertModel, GPT2LMHeadModel, ElectraModel, BertModel, AlbertModel, FunnelModel
from transformers import PreTrainedTokenizerFast, AutoTokenizer, BertTokenizerFast, ElectraTokenizerFast, FunnelTokenizerFast
from asian_bart import AsianBartTokenizer, AsianBartForConditionalGeneration
import gluonnlp as nlp
from kobert.utils import get_tokenizer
from kobert.pytorch_kobert import get_pytorch_kobert_model

device='cuda:1'

# Base Models' Paths

In [ ]:
model_paths = [
    '/home/jupyter/FINAL/albert',
    '/home/jupyter/FINAL/asbart',
    '/home/jupyter/FINAL/asbart_2',
    '/home/jupyter/FINAL/electra',
    '/home/jupyter/FINAL/funnel',
    '/home/jupyter/FINAL/funnel_2',
    '/home/jupyter/FINAL/kobart',
    '/home/jupyter/FINAL/kobart_2',
    '/home/jupyter/FINAL/kobert',
    '/home/jupyter/FINAL/kogpt2',
    '/home/jupyter/FINAL/kogpt3',
    '/home/jupyter/FINAL/kogpt3_2',
    '/home/jupyter/FINAL/mlbert',
    '/home/jupyter/FINAL/mlbert_2'
]
model_paths

# Load Backbones & Tokenizers & DataLoader

In [16]:
# load data
root = '/home/jupyter/FINAL/data/1. 실습용자료_final.txt'
data = pd.read_csv(root, sep='|', encoding='cp949')
train, test, cat2id, id2cat = preprocess(data, num_test=50000, upsample='', minimum=500, clean_fn=None, target='S', seed=5986)
test

,text,label
0,고객을 대상으로 세차장에서 자동차세차업,221
1,사업장에서 고객의요청에따라 자동차세차업,221
2,자동차부분정비 서비스 인젝터수리,221
3,"카센타 자동차의 특정부분만을 전문적 수리 경정비,타이어,내장수리,도장 등의 서비스",221
4,정비사업소 고객의 요청으로 자동차 특정부분수리,221
...,...,...
0,시설을 갖추고 사업장에서 종교라디오방송,153
1,라디오영어 FM방송을 제작하여 방송국에서 라디오영어 방송 제공,153
0,"플라스틱, 잉크 자재입고, 사출 교육용 CD",69
1,"통신및방송장비, 영상및음향기기",69


In [39]:
# load backbones and tokenizers
backbones = {}
tokenizers = {}
estimators=len(model_paths)
for path in model_paths:
    model_path = os.path.join(path, f'weights/best_loss.pth.tar')
    config_path = os.path.join(path, 'config.json')
    with open(config_path, 'r', encoding='utf-8-sig') as f:
        model_args = json.load(f)
    if model_args['model'] + '_tokenizer' not in tokenizers.keys():
        backbone, tokenizer = load_backbone_tokenizer(model_args['model'], max_len=50)
        backbones[model_args["model"]] = backbone
        tokenizers[f'{model_args["model"]}_tokenizer'] = tokenizer

using cached model. /home/jupyter/FINAL/.cache/kobart_base_tokenizer_cased_cf74400bce.zip
using cached model. /home/jupyter/FINAL/.cache/kobart_base_cased_ff4bda5738.zip
using cached model. /home/jupyter/FINAL/.cache/kobert_v1.zip
using cached model. /home/jupyter/FINAL/.cache/kobert_news_wiki_ko_cased-1087f8699e.spiece
using cached model. /home/jupyter/FINAL/.cache/kobert_news_wiki_ko_cased-1087f8699e.spiece


Some weights of the model checkpoint at bert-base-multilingual-cased were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model ch

In [40]:
print(backbones.keys())
print(tokenizers.keys())

dict_keys(['albert', 'asbart', 'electra', 'funnel', 'kobart', 'kobert', 'kogpt2', 'kogpt3', 'mlbert'])
dict_keys(['albert_tokenizer', 'asbart_tokenizer', 'electra_tokenizer', 'funnel_tokenizer', 'kobart_tokenizer', 'kobert_tokenizer', 'kogpt2_tokenizer', 'kogpt3_tokenizer', 'mlbert_tokenizer'])


In [41]:
# load dataloader
test_set = EnsembleDataset(test['text'].tolist(), test['label'].tolist(), **tokenizers, max_len=50)
test_loader = DataLoader(test_set, batch_size=768, num_workers=4, shuffle=False, pin_memory=False)

# Test

In [47]:
num_classes=225
estimators=len(model_paths)

base_predictions = defaultdict(lambda: None) # Test data에 대한 base model의 예측(소프트맥스 확률값)을 저장
labels = None # Test data label 저장

# 성능 평가 도구 : Evaluator
eval_base_models = defaultdict(lambda: Evaluator(num_classes)) # 개별 base model 성능 평가
eval_mean = Evaluator(num_classes) # mean aggregation 결과 평가
eval_vote = Evaluator(num_classes) # vote aggregation 결과 평가

# Test
for e, path in enumerate(model_paths):
    # paths
    model_path = os.path.join(path, f'weights/best_loss.pth.tar')
    config_path = os.path.join(path, 'config.json')
    
    # info from config.json
    with open(config_path, 'r', encoding='utf-8-sig') as f:
        model_args = json.load(f)
    loss_type = model_args['loss']
    model_type = model_args['model']
#     model_name = model_args['model']
    model_name = path.split(os.sep)[-1]
#     model_name = '_'.join([str(e), model_type, path.split(os.sep)[-1]])
    
    # build a base model
    backbone = backbones[model_type] # load backbone
    model = load_model(model_type=model_type,
                       backbone=copy(backbone),
                       num_classes=num_classes,
                       num_layers=model_args['n_layers'], 
                       dr_rate=model_args['dr_rate'],
                       bias=True,
                       batchnorm=model_args['batchnorm'],
                       layernorm=model_args['layernorm'])
    model.load_state_dict(torch.load(model_path, map_location='cpu')['state_dict'])
    model = model.to(device)
    model.eval()
    
    # Base Model Test
    with torch.no_grad():
        for inputs, label in tqdm(test_loader, total=len(test_loader), ncols=100,
                                  desc=f'{e}/{len(model_paths)-1} {model_name}'):
            # pick base model input from inputs
            _inputs = {}
            _inputs['input_ids'] = inputs[model_type][:, 0].to(device, non_blocking=True)
            _inputs['attention_mask'] = inputs[model_type][:, 1].to(device, non_blocking=True)
            if 'bart' not in model_type:
                _inputs['token_type_ids'] = inputs[model_type][:, 2].to(device, non_blocking=True)
            
            # forward
            output = model(**_inputs)
            pred = F.softmax(output, dim=1)
            
            # store base predictions
            if base_predictions[model_name] is None:
                base_predictions[model_name] = pred.clone().cpu()
            else:
                base_predictions[model_name] = torch.cat([base_predictions[model_name], pred.clone().cpu()], dim=0)
            # store labels
            if e==0:
                if labels is None:
                    labels = label.cpu()
                else:
                    labels = torch.cat([labels, label.cpu()], dim=0)
            
            # update batch predictions
            eval_base_models[model_name].update(pred.clone().argmax(1).cpu(), label)
    # log a base model performance
    eval_base_models[model_name].compute()
    print(f'\tacc: {round(eval_base_models[model_name].acc, 6)}\tpc: {round(eval_base_models[model_name].macro_pc, 6)}\trc: {round(eval_base_models[model_name].macro_rc, 6)}\tf1: {round(eval_base_models[model_name].macro_f1, 6)}')

0/13 albert: 100%|██████████████████████████████████████████████████| 66/66 [00:29<00:00,  2.20it/s]


	acc: 0.893217	pc: 0.773534	rc: 0.737759	f1: 0.745843


1/13 asbart: 100%|██████████████████████████████████████████████████| 66/66 [01:24<00:00,  1.28s/it]


	acc: 0.894479	pc: 0.776831	rc: 0.732162	f1: 0.74218


2/13 asbart_2: 100%|████████████████████████████████████████████████| 66/66 [01:25<00:00,  1.29s/it]


...

	acc: 0.903988	pc: 0.798322	rc: 0.754658	f1: 0.767265


12/13 mlbert: 100%|█████████████████████████████████████████████████| 66/66 [00:23<00:00,  2.77it/s]


	acc: 0.89614	pc: 0.768502	rc: 0.749519	f1: 0.750923


13/13 mlbert_2: 100%|███████████████████████████████████████████████| 66/66 [00:23<00:00,  2.77it/s]

	acc: 0.890475	pc: 0.777429	rc: 0.733732	f1: 0.744785


In [49]:
# mean aggregation
base_model_outputs = torch.stack(list(base_predictions.values()))
ensemble_output_mean = torch.mean(base_model_outputs, dim=0)
pred_mean = ensemble_output_mean.argmax(1).cpu()
eval_mean.update(pred_mean.tolist(), labels.tolist())
eval_mean.compute()
# vote aggregation
ensemble_output_vote = vote(base_model_outputs, dim=2)
pred_vote = ensemble_output_vote.sum(0).argmax(1).cpu()
eval_vote.update(pred_vote.tolist(), labels.tolist())
eval_vote.compute()

# log aggregation result
print('\n\nResult')
print(f'mean agg|\tacc: {round(eval_mean.acc, 6)}\tpc: {round(eval_mean.macro_pc, 6)}\trc: {round(eval_mean.macro_rc, 6)}\tf1: {round(eval_mean.macro_f1, 6)}')
print(f'vote agg|\tacc: {round(eval_vote.acc, 6)}\tpc: {round(eval_vote.macro_pc, 6)}\trc: {round(eval_vote.macro_rc, 6)}\tf1: {round(eval_vote.macro_f1, 6)}')
# log base models' result again
for e, (model_name, eval_base_model) in enumerate(eval_base_models.items()):
    print(f'{e} {model_name} | \tacc: {round(eval_base_model.acc, 6)}\tpc: {round(eval_base_model.macro_pc, 6)}\trc: {round(eval_base_model.macro_rc, 6)}\tf1: {round(eval_base_model.macro_f1, 6)}')



Result
mean agg|	acc: 0.912136	pc: 0.816902	rc: 0.78041	f1: 0.791835
vote agg|	acc: 0.912156	pc: 0.817654	rc: 0.781789	f1: 0.792392
0 albert | 	acc: 0.893217	pc: 0.773534	rc: 0.737759	f1: 0.745843
1 asbart | 	acc: 0.894479	pc: 0.776831	rc: 0.732162	f1: 0.74218
2 asbart_2 | 	acc: 0.89594	pc: 0.787935	rc: 0.741071	f1: 0.753943
3 electra | 	acc: 0.900024	pc: 0.775634	rc: 0.739965	f1: 0.748075
4 funnel | 	acc: 0.903447	pc: 0.778733	rc: 0.763929	f1: 0.765086
5 funnel_2 | 	acc: 0.901746	pc: 0.778926	rc: 0.748958	f1: 0.755827
6 kobart | 	acc: 0.896661	pc: 0.780303	rc: 0.751514	f1: 0.757613
7 kobart_2 | 	acc: 0.896541	pc: 0.761119	rc: 0.738662	f1: 0.742271
8 kobert | 	acc: 0.898723	pc: 0.771965	rc: 0.743887	f1: 0.7515
9 kogpt2 | 	acc: 0.891196	pc: 0.792242	rc: 0.716439	f1: 0.737997
10 kogpt3 | 	acc: 0.904248	pc: 0.782837	rc: 0.775835	f1: 0.773007
11 kogpt3_2 | 	acc: 0.903988	pc: 0.798322	rc: 0.754658	f1: 0.767265
12 mlbert | 	acc: 0.89614	pc: 0.768502	rc: 0.749519	f1: 0.750923
13 mlbert_2 | 

# Best 조합

In [53]:
from itertools import combinations
import time

n = 1
model_predictions = defaultdict(list)
for model_name, predictions in base_predictions.items():
    model_type = model_name.split('_')[0]
    model_predictions[model_type].append((model_name, predictions))
    
model_coef={}
eval_test = Evaluator(num_classes)
predictions = []
model_names = []
for model_type, model_preds in model_predictions.items():
    combs = list(combinations(model_preds, n))
    for comb in combs:
        model_name, preds = zip(*comb)
        predictions += preds
        model_names.append(model_name)
agg_pred = torch.stack(predictions)
ensemble_output_mean = torch.mean(agg_pred, dim=0)
pred_mean = ensemble_output_mean.argmax(1).cpu()
eval_test.update(pred_mean.tolist(), labels.tolist())
eval_test.compute()
print(eval_test.acc, eval_test.macro_pc, eval_test.macro_rc, eval_test.macro_f1)

0.9121356502242153 0.8169016274378694 0.7804104475723467 0.7918351755669315


In [54]:
from itertools import combinations

model_coef={}
for n in range(1, len(base_predictions)+1):
    combs = list(combinations(sorted(base_predictions.items()), n))
    for comb in tqdm(combs, total=len(combs), desc=f'{n}개 조합', ncols=100):
        models, predictions = zip(*comb)
        eval_test = Evaluator(num_classes)
        agg_pred = torch.stack(predictions)
        ensemble_output_mean = torch.mean(agg_pred, dim=0)
        pred_mean = ensemble_output_mean.argmax(1).cpu()
        eval_test.update(pred_mean.tolist(), labels.tolist())
        eval_test.compute()
        model_coef['|'.join(models)] = (eval_test.acc, eval_test.macro_f1)
    

14개 조합: 100%|██████████████████████████████████████████████████████| 1/1 [00:00<00:00,  6.24it/s]


In [59]:
# acc 기준 조합 랭킹
rank = 5
model_acc = {mt: acc for mt, (acc, f1) in model_coef.items()}
sorted(zip(map(lambda x: round(x, 6), model_acc.values()), model_acc.keys()), reverse=True)[:rank]

[(0.912636,
  'asbart_2|funnel|funnel_2|kobart_2|kobert|kogpt2|kogpt3|kogpt3_2|mlbert|mlbert_2'),
 (0.912616,
  'asbart_2|funnel|funnel_2|kobart|kobart_2|kobert|kogpt2|kogpt3|kogpt3_2|mlbert|mlbert_2'),
 (0.912616,
  'asbart_2|electra|funnel_2|kobert|kogpt2|kogpt3|kogpt3_2|mlbert|mlbert_2'),
 (0.912576,
  'asbart|asbart_2|funnel|funnel_2|kobart_2|kobert|kogpt2|kogpt3_2|mlbert|mlbert_2'),
 (0.912576,
  'albert|asbart|asbart_2|funnel_2|kobart_2|kobert|kogpt2|kogpt3|kogpt3_2|mlbert|mlbert_2')]

In [60]:
# f1 기준 조합 랭킹
model_f1 = {mt: f1 for mt, (acc, f1) in model_coef.items()}
sorted(zip(map(lambda x: round(x, 6), model_f1.values()), model_f1.keys()), reverse=True)[:rank]

[(0.799122,
  'albert|asbart|asbart_2|funnel|kobert|kogpt2|kogpt3|mlbert|mlbert_2'),
 (0.798517,
  'albert|asbart|asbart_2|electra|funnel|funnel_2|kobart|kobert|kogpt2|kogpt3|mlbert_2'),
 (0.798496,
  'albert|asbart|asbart_2|electra|funnel|kobart|kobert|kogpt2|kogpt3|mlbert_2'),
 (0.798383, 'asbart|asbart_2|funnel|kobert|kogpt3|kogpt3_2|mlbert_2'),
 (0.798105, 'albert|asbart|asbart_2|funnel|kobert|kogpt2|kogpt3|mlbert')]

In [71]:
# acc, f1 가중합 기준 조합 랭킹
w_acc = 0.6
w_f1 = 1-w_acc
model_coef2 = {mt: acc*w_acc + f1*w_f1 for mt, (acc, f1) in model_coef.items()}
sorted(zip(map(lambda x: round(x, 6), model_coef2.values()), model_coef2.keys()), reverse=True)[:rank]

[(0.867038,
  'albert|asbart|asbart_2|funnel|kobert|kogpt2|kogpt3|mlbert|mlbert_2'),
 (0.866478, 'asbart|asbart_2|funnel|kobert|kogpt3|kogpt3_2|mlbert_2'),
 (0.866469,
  'asbart|asbart_2|electra|funnel|funnel_2|kobart|kobert|kogpt2|kogpt3|kogpt3_2|mlbert_2'),
 (0.866447,
  'albert|asbart|asbart_2|electra|funnel|funnel_2|kobert|kogpt2|kogpt3|mlbert|mlbert_2'),
 (0.866438, 'asbart|asbart_2|funnel_2|kobert|kogpt2|kogpt3|kogpt3_2|mlbert')]

# Inference

In [84]:
class arguments():
    def __init__(self):
        self.root='/home/jupyter/FINAL/data/2. 모델개발용자료_final.txt'
        self.batch_size=728
        self.device='cuda'
    
args = arguments()

In [72]:
model_paths = [
    '/home/jupyter/FINAL/albert',
    '/home/jupyter/FINAL/asbart',
    '/home/jupyter/FINAL/asbart_2',
    '/home/jupyter/FINAL/funnel',
    '/home/jupyter/FINAL/kobert',
    '/home/jupyter/FINAL/kogpt2',
    '/home/jupyter/FINAL/kogpt3',
    '/home/jupyter/FINAL/mlbert',
    '/home/jupyter/FINAL/mlbert_2'
]

['/home/jupyter/FINAL/albert',
 '/home/jupyter/FINAL/asbart',
 '/home/jupyter/FINAL/asbart_2',
 '/home/jupyter/FINAL/funnel',
 '/home/jupyter/FINAL/kobert',
 '/home/jupyter/FINAL/kogpt2',
 '/home/jupyter/FINAL/kogpt3',
 '/home/jupyter/FINAL/mlbert',
 '/home/jupyter/FINAL/mlbert_2']

In [78]:
with open(os.path.join('/home/jupyter/FINAL/mlbert_2/id2cat.json'), 'r') as f:
    id2cat = json.load(f)

In [85]:
data = pd.read_csv(args.root, sep='|', encoding='cp949')
doc_id, doc = data.index.tolist(), data[['text_obj', 'text_mthd', 'text_deal']].fillna('')
doc = doc.apply(lambda x: ' '.join(x), axis=1).tolist()

In [86]:
# Ensemble Data Loader
test_set = EnsembleDataset(doc, doc_id, **tokenizers,
                           max_len=50)
test_loader = DataLoader(test_set, batch_size=512, num_workers=4,
                         shuffle=False, pin_memory=False)

In [87]:
num_classes=225

estimators=len(model_paths)

base_model_outputs = []

doc_ids = None

for e, path in enumerate(model_paths):
    model_path = os.path.join(path, f'weights/best_loss.pth.tar')
    config_path = os.path.join(path, 'config.json')
    with open(config_path, 'r', encoding='utf-8-sig') as f:
        model_args = json.load(f)
    loss_type = model_args['loss']
    model_type = model_args['model']
    if 'sample_dist' in model_args.keys():
        dist = model_args['sample_dist']
        model_name = '_'.join([model_type, loss_type, dist])
    else:
        model_name = '_'.join([model_type, loss_type])
    backbone = backbones[model_args['model']]
    model = load_model(model_type=model_args['model'],
                       backbone=copy(backbone),
                       num_classes=num_classes,
                       num_layers=model_args['n_layers'], 
                       dr_rate=model_args['dr_rate'],
                       bias=True,
                       batchnorm=model_args['batchnorm'],
                       layernorm=model_args['layernorm'])
    model.load_state_dict(torch.load(model_path, map_location='cpu')['state_dict'])
    model = model.to(device)
    model.eval()
    
    base_model_output = None
    with torch.no_grad():
        for inputs, doc_id in tqdm(test_loader, total=len(test_loader), ncols=100, desc=f'{e}/{len(model_paths)-1} {model_name}'):
            _inputs = {}
            _inputs['input_ids'] = inputs[model_type][:, 0].to(device, non_blocking=True)
            _inputs['attention_mask'] = inputs[model_type][:, 1].to(device, non_blocking=True)
            if 'bart' not in model_type:
                _inputs['token_type_ids'] = inputs[model_type][:, 2].to(device, non_blocking=True)
            output = model(**_inputs)
            pred = F.softmax(output, dim=1)
            if base_model_output is None:
                base_model_output = pred.clone().cpu()
            else:
                base_model_output = torch.cat([base_model_output, pred.clone().cpu()], dim=0)
            if e==0:
                if doc_ids is None:
                    doc_ids = doc_id
                else:
                    doc_ids = torch.cat([doc_ids, doc_id])
    base_model_outputs.append(base_model_output)
base_model_outputs = torch.stack(base_model_outputs)


ensemble_output_mean = torch.mean(base_model_outputs, dim=0)
output_cat = list(map(lambda x: id2cat[str(x)], ensemble_output_mean.argmax(1).cpu().tolist()))
data[['digit_1', 'digit_2', 'digit_3']] = pd.DataFrame(list(zip(*[map(lambda x: x[0], output_cat),
                                                                  map(lambda x: x[1:3], output_cat),
                                                                  map(lambda x: x[3:], output_cat)])))

8/9 mlbert_FCE: 100%|█████████████████████████████████████████████| 196/196 [00:47<00:00,  4.17it/s]


In [93]:
data.drop(['Unnamed: 0'],inplace=True,axis=1)

In [96]:
data.to_csv('./submit_0413_final.csv', index=False, encoding='utf-8-sig')

In [95]:
data[:20]

,AI_id,digit_1,digit_2,digit_3,text_obj,text_mthd,text_deal
0,id_000001,I,56,561,치킨전문점에서,고객의주문에의해,치킨판매
1,id_000002,G,46,466,산업공구,다른 소매업자에게,철물 수공구
2,id_000003,S,94,949,절에서,신도을 대상으로,불교단체운영
3,id_000004,S,95,952,영업장에서,고객요구로,자동차튜닝
4,id_000005,I,56,562,실내포장마차에서,접객시설을 갖추고,"소주,맥주제공"
5,id_000006,C,18,181,"철,아크릴,포맥스",스크린인쇄,명판
6,id_000007,I,56,561,음식점,접객시설가지고,조개구이판매
7,id_000008,C,25,259,스테인레스를,프레스가공하여제조,주방용품
8,id_000009,S,95,952,수리,서비스센터에서,전문수리 수입차
9,id_000010,C,25,259,"약품(화공), 미싱","완성품입고, 수선",NaN
